In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Model is using 8000 positive and 8000 negative
# 4000 tweets are general positive while 4000 are airlines for positive sentiments
# 2400 are disaster  and news tweets, 3000 tweets are negative airline sentiments and 2500 general negative

Using TensorFlow backend.


In [2]:
data = pd.read_csv('clean_tweet_model9.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [3]:
test_data = pd.read_csv('news_tests.csv')
# Keeping only the neccessary columns
test_data = test_data[['text','sentiment']]

In [7]:
data['text'] = data['text'].apply(lambda x: str(x))
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)
    
max_features = 3000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X_train = tokenizer.texts_to_sequences(data['text'].values)
X_train = pad_sequences(X_train)

22216
15798


In [8]:
test_data['text'] = test_data['text'].apply(lambda x: str(x))
test_data['text'] = test_data['text'].apply(lambda x: x.lower())
test_data['text'] = test_data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(test_data[ test_data['sentiment'] == 'Positive'].size)
print(test_data[ test_data['sentiment'] == 'Negative'].size)
    
tokenizer_test = Tokenizer(num_words=max_features, split=' ')
X_test = tokenizer.texts_to_sequences(test_data['text'].values)
X_test = pad_sequences(X_test, maxlen=X_train.shape[1])

1992
2020


In [9]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 222, 128)          384000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 222, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 639,194
Trainable params: 639,194
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
Y_train = pd.get_dummies(data['sentiment']).values
Y_test = pd.get_dummies(test_data['sentiment']).values
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(19007, 222) (19007, 2)
(2006, 222) (2006, 2)


In [11]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 12, batch_size=batch_size, verbose = 2)

Epoch 1/12
 - 458s - loss: 0.4834 - acc: 0.7603
Epoch 2/12
 - 467s - loss: 0.3751 - acc: 0.8339
Epoch 3/12
 - 419s - loss: 0.3386 - acc: 0.8511
Epoch 4/12
 - 433s - loss: 0.3081 - acc: 0.8673
Epoch 5/12
 - 388s - loss: 0.2880 - acc: 0.8730
Epoch 6/12
 - 448s - loss: 0.2679 - acc: 0.8853
Epoch 7/12
 - 456s - loss: 0.2505 - acc: 0.8924
Epoch 8/12
 - 428s - loss: 0.2338 - acc: 0.9005
Epoch 9/12
 - 412s - loss: 0.2138 - acc: 0.9117
Epoch 10/12
 - 409s - loss: 0.1985 - acc: 0.9175
Epoch 11/12
 - 464s - loss: 0.1838 - acc: 0.9239
Epoch 12/12
 - 401s - loss: 0.1687 - acc: 0.9318


In [12]:
X_validate = X_test
Y_validate = Y_test
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.08
acc: 0.98


In [17]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
        
    if np.argmax(result) != np.argmax(Y_validate[x]):
        print(test_data.values[x][0])
        print(test_data.values[x][1])
     
    
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

we do love a good storm you double tap if you agree
Positive
it s so dry here could hardly breathe unbearable
Negative
we are doing our bit southafrica with a little help from our friends spar this is the st of borehole projects we hope to complete in kwazulunatal
Positive
caption this throwback to summer in california people
Positive
salvaged cut and stacked
Negative
harmful effects of landslides download pdf and find out more only at landslides
Negative
there possibly couldn t be any road here could there anyways please stay safe wherever you are be sure it s safe to venture out before you do
Positive
you need a new hobby wanna train for a triathlon with me
Positive
teamviewer is entertaining
Positive
modest mouse is the shit
Positive
you guys must be having a ball looks like you re getting pretty buggered though must be hard
Positive
in theory that is supposed to work but in reality not so much i keep hitting that x though
Negative
it has been less than an hour now
Positive
ate too 

In [15]:
txt = ['major delay in traffic due to accident']
#vectorizing the tweet by the pre-fitted tokenizer instance
txt = tokenizer.texts_to_sequences(txt)
#padding the tweet to have exactly the same shape as `embedding_2` input
txt = pad_sequences(txt, maxlen=222, dtype='int32', value=0)
# print(twt)
sentiment = model.predict(txt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

negative


In [16]:
model.save('my_model10.h5')